Librerías

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

from sklearn.metrics import mean_absolute_error, mean_squared_error

from prophet import Prophet
#from prophet.diagnostics import generate_cutoffs, cross_validation, performance_metrics
from prophet.plot import plot_plotly
import plotly.graph_objects as go

import time


- Ce carga un subdataset con los datos necesarios para el modelo de ML

In [ ]:
# Cargar los datos
df = pd.read_csv(r"..\datasets\2. Depurados\TLC Aggregated Data\ML_TS_Input.csv")
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# Obtener el valor máximo de la columna 'date'
max_date = df['date'].max()
# Calcular la fecha de diciembre dentro de 5 años
future_date = datetime(max_date.year + 5, 12, 1)
# Calcular la cantidad de meses entre ambas fechas
months_difference = (future_date.year - max_date.year) * 12 + (future_date.month - max_date.month)

In [ ]:
# Función para guardar el forecast en un archivo CSV
def guardar_predicciones(forecast, nombre_archivo="predicciones.csv"):
    try:
        forecast.to_csv(nombre_archivo, index=False)
        print(f"Predicciones guardadas en {nombre_archivo}")
    except Exception as e:
        print(f"Error al guardar las predicciones: {e}")

In [ ]:
def graficar_original(df_prophet, column_name):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_prophet["ds"], y=df_prophet["y"], marker=dict(symbol='circle', color='royalblue')))
    fig.layout.update(title_text="Datos históricos", yaxis_title=f"{column_name}", xaxis_rangeslider_visible=True)
    fig.show()

In [ ]:
def graficar_predicción(df_prophet, column_name, forecast,industry_type):

    # Ajustar valores negativos en el forecast
    forecast['yhat'] = forecast['yhat'].clip(lower=0)
    forecast['yhat_lower'] = forecast['yhat_lower'].clip(lower=0)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_prophet["ds"], y=df_prophet["y"], name='Datos Históricos', marker=dict(symbol='circle', color='royalblue')))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicción', marker=dict(symbol='diamond', color='green')))
    fig.add_trace(go.Scatter(x=forecast['ds'].tolist() + forecast['ds'][::-1].tolist(), 
                            y=forecast['yhat_upper'].tolist() + forecast['yhat_lower'][::-1].tolist(), 
                            fill='toself', fillcolor='rgba(255, 255, 255, 0.2)', 
                            line=dict(color='rgba(255, 255, 255, 0)'), name='Intervalo de predicción'))
    fig.layout.update(xaxis_title='Fecha', yaxis_title=column_name, title_text=f"Predicción para {column_name} ({industry_type})", 
                    xaxis_rangeslider_visible=True)
    fig.show()

# Análisis para cada variable y cada industria

In [ ]:
# Selección de períodos y frecuencia para predicción
industry_types = df['industry'].unique()
columns_to_predict = ['total_trips', 'unique_vehicles', 'total_amount', 'avg_trip_distance', 'total_co2_emission']
periodos = 12
frecuencia = "MS"

In [ ]:
df.head()

,date,industry,total_trips,unique_vehicles,total_amount,avg_trip_distance,avg_hours_per_day_per_driver,total_co2_emission,days_in_month
0,2021-01-01,FHV - High Volume,11902481,47594,1.917688e+08,3.64,6.800,17332.2978,31
1,2021-01-01,Yellow Taxi,1365612,4625,2.135305e+07,2.43,7.300,1326.4672,31
2,2021-01-01,Green Taxi,76477,982,1.748679e+06,3.46,4.000,105.8211,31
3,2021-01-01,FHV - Other,1142350,10128,NaN,3.63,3.800,1660.0704,31
4,2021-01-01,Total Mercado,14486920,63329,2.148706e+08,3.29,5.475,20424.6565,31


In [ ]:
# Función para cargar los datos y filtrar la serie de tiempo seleccionada
def cargar_y_preparar_datos(df, industry_type, column_name):
    """
    Filtra y prepara los datos para Prophet según el tipo de industria y la columna seleccionada.
    """
    df_filtered = df[df['industry'] == industry_type][['date', column_name]].copy()
    df_filtered.columns = ['ds', 'y']  # Renombrar columnas para Prophet
    df_filtered['ds'] = pd.to_datetime(df_filtered['ds'])  # Asegurar formato de fecha
    return df_filtered if not df_filtered['y'].isnull().all() else None

In [ ]:
def evaluate_model(df_forecast, df_prophet_test):
    # Extraer valores reales y predichos
    y_true = df_prophet_test ['y'].values
    y_pred = df_forecast[df_forecast['ds'] > pd.to_datetime("2023-08-01", format='%Y-%m-%d')]['yhat'].values

    # Calcular métricas
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    smape = 100 / len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    
    # print(" ")  
    # print("Resultados de validación cruzada:")    
    # print(f'Mean Absolute Error: {mae:.2f}')
    # print(f'Mean Absolute Percentage Error : {mape:.2f}')
    # print(f'Symmetric Mean Absolute Percentage Errorr : {smape:.2f}')
    # print(f'Mean Squared Error: {mse:.2f}')
    # print(f'Root Mean Squared Error: {rmse:.2f}')
    # print(" ")  

    return {'mae': mae, 'mape': mape, 'smape': smape, 'mse': mse, 'rmse': rmse}


In [ ]:
def pronóstico_con_grid_search(df_prophet, column_name, industry_type, periodos, frecuencia):
    """
    Realiza Grid Search para encontrar los mejores hiperparámetros de Prophet y calcula el error.
  
    Parameters that can be tuned

    - changepoint_prior_scale: This is probably the most impactful parameter. It determines the flexibility of the trend, and in particular how 
      much the trend changes at the trend changepoints. As described in this documentation, if it is too small, the trend will be underfit and 
      variance that should have been modeled with trend changes will instead end up being handled with the noise term. If it is too large, the trend 
      will overfit and in the most extreme case you can end up with the trend capturing yearly seasonality. The default of 0.05 works for many time series, 
      but this could be tuned; a range of [0.001, 0.5] would likely be about right. Parameters like this (regularization penalties; this is effectively a 
      lasso penalty) are often tuned on a log scale.

    - seasonality_prior_scale: This parameter controls the flexibility of the seasonality. Similarly, a large value allows the seasonality to fit large fluctuations, 
      a small value shrinks the magnitude of the seasonality. The default is 10., which applies basically no regularization. That is because we very rarely 
      see overfitting here (there’s inherent regularization with the fact that it is being modeled with a truncated Fourier series, so it’s essentially low-pass 
      filtered). A reasonable range for tuning it would probably be [0.01, 10]; when set to 0.01 you should find that the magnitude of seasonality is forced to be 
      very small. This likely also makes sense on a log scale, since it is effectively an L2 penalty like in ridge regression.

    - seasonality_mode: Options are ['additive', 'multiplicative']. Default is 'additive', but many business time series will have multiplicative seasonality. 
      This is best identified just from looking at the time series and seeing if the magnitude of seasonal fluctuations grows with the magnitude of the time series 
      (see the documentation here on multiplicative seasonality), but when that isn’t possible, it could be tuned.

      
    """

    grid_search = {
        'changepoint_prior_scale': [0.01, 0.05, 0.1, 0.15, 0.2],
        'seasonality_prior_scale': [0.01, 0.05, 0.1, 1.0, 5.0, 10.0, 15.0, 20.0],  
        'seasonality_mode': ['additive', 'multiplicative']
    }
    
    resultados = []  
    
    df_prophet_train = df_prophet[df_prophet['ds']<= pd.to_datetime("2023-08-01", format='%Y-%m-%d')]
    df_prophet_test = df_prophet[df_prophet['ds'] >=  pd.to_datetime("2023-09-01", format='%Y-%m-%d')]

    # Grid Search
    for cps in grid_search['changepoint_prior_scale']:
        for sps in grid_search['seasonality_prior_scale']:
            for sm in grid_search['seasonality_mode']:
                        start_time = time.time()
                      
                        print(f"Evaluación combinación: CPS={cps}, SPS={sps}, SM={sm}") #FO={fo}

                        # Crear y entrenar modelo Prophet con los parámetros actuales
                        model = Prophet(changepoint_prior_scale=cps, seasonality_prior_scale=sps, seasonality_mode=sm, yearly_seasonality=True, weekly_seasonality=False)
                        
                        """Prophet includes yearly, weekly, and daily seasonality by default for daily data. 
                        For sub-daily data, only daily seasonality is included by default."""
                        #model.add_seasonality(name='monthly', period=12, fourier_order=3)                        
                        model.fit(df_prophet_train)

                        # Crear el futuro dataframe con frecuencia específica
                        future = model.make_future_dataframe(periods=periodos, freq=frecuencia)
                        forecast = model.predict(future)
                        #graficar_predicción(df_prophet, column_name, forecast, industry_type)

                        # Evaluar modelo
                        metrics = evaluate_model(forecast,df_prophet_test)
                        if metrics is None:
                            continue
                        
                        elapsed_time = time.time() - start_time
                        resultados.append({
                            'changepoint_prior_scale': cps,
                            'seasonality_prior_scale': sps,
                            'seasonality_mode': sm,
                            **metrics,
                            'training time': elapsed_time
                        })

                        # Crear DataFrame con todos los resultados
                        df_resultados = pd.DataFrame(resultados)

                        # Identificar los mejores parámetros según el menor MAE
                        if not df_resultados.empty:
                            mejor_modelo = df_resultados.loc[df_resultados['mae'].idxmin()]
                        else:
                            mejor_modelo = None

    return df_resultados, mejor_modelo

In [ ]:
# Iterar sobre todas las combinaciones de industria y columna
mejores_resultados = {}
todos_los_resultados = pd.DataFrame()

for industry_type in industry_types:
    for column_name in columns_to_predict:
        # Preparar los datos
        print(f"Procesando: {industry_type}, columna: {column_name}")
        df_prophet = cargar_y_preparar_datos(df, industry_type, column_name)

        if df_prophet is not None and not df_prophet.empty and df_prophet['y'].notnull().all():

            # graficar_original(df_prophet, column_name)

            # Ejecutar Grid Search
            df_resultados, mejor_modelo= pronóstico_con_grid_search(df_prophet, column_name,industry_type, periodos=12, frecuencia="MS")

            # Guardar resultados de Grid Search
            if not df_resultados.empty:
                # Añadir columnas 'industry' y 'column' al DataFrame df_resultados
                df_resultados['industry'] = industry_type
                df_resultados['column'] = column_name

                # Reordenar las columnas del DataFrame
                df_resultados = df_resultados[['industry', 'column', 'changepoint_prior_scale', 'seasonality_prior_scale', 
                                'seasonality_mode', 'mae', 'mape', 'smape', 'mse', 'rmse', 'training time']] 


                # Concatenar df_resultados con todos_los_resultados
                todos_los_resultados = pd.concat([todos_los_resultados, df_resultados], ignore_index=True)

            # Guardar mejores resultados
            if mejor_modelo is not None:
                mejores_resultados[f"{industry_type}/{column_name}"] = mejor_modelo.to_dict()         

# Guardar todos los resultados en un archivo
todos_los_resultados.to_csv("todos_los_resultados_modelos.csv", index=False)

# Guardar el mejor modelo por combinación en otro archivo
df_mejores_resultados = pd.DataFrame(mejores_resultados).T.reset_index()
df_mejores_resultados[['industry', 'column']] = df_mejores_resultados['index'].str.split('/', expand=True)
df_mejores_resultados = df_mejores_resultados.drop(columns=['index'])
df_mejores_resultados = df_mejores_resultados[['industry', 'column', 'changepoint_prior_scale', 'seasonality_prior_scale', 
                                               'seasonality_mode', 'mae', 'mape', 'smape', 'mse', 'rmse', 'training time']] 
df_mejores_resultados.to_csv("mejores_modelos.csv", index=False)

22:55:17 - cmdstanpy - INFO - Chain [1] done processing
22:55:18 - cmdstanpy - INFO - Chain [1] start processing


Evaluación combinación: CPS=0.24, SPS=11.0, SM=additive


22:55:50 - cmdstanpy - INFO - Chain [1] done processing
22:55:50 - cmdstanpy - INFO - Chain [1] start processing


Evaluación combinación: CPS=0.24, SPS=11.0, SM=multiplicative


22:56:17 - cmdstanpy - INFO - Chain [1] done processing
22:56:17 - cmdstanpy - INFO - Chain [1] start processing


...

00:46:53 - cmdstanpy - INFO - Chain [1] done processing
00:46:54 - cmdstanpy - INFO - Chain [1] start processing


Evaluación combinación: CPS=0.25, SPS=15.0, SM=additive


00:47:18 - cmdstanpy - INFO - Chain [1] done processing
00:47:18 - cmdstanpy - INFO - Chain [1] start processing


Evaluación combinación: CPS=0.25, SPS=15.0, SM=multiplicative


00:47:43 - cmdstanpy - INFO - Chain [1] done processing
